In [10]:
import tensorflow as tf
import tensorflow.keras as keras

import numpy as npc

# Char-NNN 

In [6]:
# shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
shakespeare_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()
print(shakespeare_text[:148])

1122304/1115394 [==============================] - 0s 0us/step
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [11]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

tokenizer.texts_to_sequences(["First"])
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))

np.random.seed(42)
tf.random.set_seed(42)

batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

dataset = dataset.prefetch(1)

for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)



(32, 100, 39) (32, 100)


In [12]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=10)

Epoch 1/10
  552/31370 [..............................] - ETA: 2:23:21 - loss: 2.6968

KeyboardInterrupt: 

In [ ]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

def next_char(text, temperature=1):
    # Plus la température est élevée et plus les différentes classes auront la même proba d'être choisie. si la température est basse, les classes les moins probables auront encore moins de chance d'être choisies.
    # Cela vient de la formule du softmax:  P(j) = exp(logit_j / T) / sum_i: exp(logit_i / T) 
    # Pour T grand: P(j) équivalent à 1 / sum_i: 1 = 1/N
    # Pour T petit: logit_j / T va écraser les autres s'il est le plus grand. 
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

tf.random.set_seed(42)
next_char("How are yo", temperature=1)

# Char-NN avec état

In [ ]:
# Pour chaque batch, le n-ème élément doit être la continuation du n-ème élément précédent, donc on ne peut pas shuffle.
# Une idée simple est simplement de considérer un batch comme 1 élément et de les prendre au fur et à mesure, sans shuffle
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.repeat().batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

# A la fin de chaque epoch, une fois qu'on a fait le tour sur le dataset, il faut ré-initialiser les states, car on repart depuis de début
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
steps_per_epoch = train_size // batch_size // n_steps
history = model.fit(dataset, steps_per_epoch=steps_per_epoch, epochs=50,
                    callbacks=[ResetStatesCallback()])

# Analyse de sentiments

In [13]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

%|█████▊    | 14655/25000 [00:04<00:02, 4089.79 examples/s]
Generating train examples...:  99%|█████████▉| 24838/25000 [00:07<00:00, 4420.01 examples/s]
                                                                                            
Generating test examples...:  99%|█████████▊| 24650/25000 [00:06<00:00, 3936.66 examples/s]
                                                                                           
Generating unsupervised examples...:  99%|█████████▉| 49577/50000 [00:14<00:00, 4643.21 examples/s]
                                                                                                   
Shuffling imdb_reviews-unsupervised.tfrecord...:   0%|          | 0/50000 [00:00<?, ? examples/s]
Shuffling imdb_reviews-unsupervised.tfrecord...:  52%|█████▏    | 26176/50000 [00:00<00:00, 261757.13 examples/s]
Dataset imdb_reviews downloaded and prepared to /home/johnny/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [16]:
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

for X_batch, y_batch in datasets["train"].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    # On remplace les mises à la ligne par des espaces
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    # On récupère chaque mot en splittant sur les espaces.
    X_batch = tf.strings.split(X_batch)
    # On met un padding pour que toutes les instances ont la même longueur
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch
preprocess(X_batch, y_batch)

Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0 = Negative

Review: I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0 = Negative



(<tf.Tensor: shape=(2, 53), dtype=string, numpy=
 array([[b'This', b'was', b'an', b'absolutely', b'terrible', b'movie',
         b"Don't", b'be', b'lured', b'in', b'by', b'Christopher',
         b'Walken', b'or', b'Michael', b'Ironside', b'Both', b'are',
         b'great', b'actors', b'but', b'this', b'must', b'simply', b'be',
         b'their', b'worst', b'role', b'in', b'history', b'Even',
         b'their', b'great', b'acting', b'could', b'not', b'redeem',
         b'this', b"movie's", b'ridiculous', b'storyline', b'This',
         b'movie', b'is', b'an', b'early', b'nineties', b'US',
         b'propaganda', b'pi', b'<pad>', b'<pad>', b'<pad>'],
        [b'I', b'have', b'been', b'known', b'to', b'fall', b'asleep',
         b'during', b'films', b'but', b'this', b'is', b'usually', b'due',
         b'to', b'a', b'combination', b'of', b'things', b'including',
         b'really', b'tired', b'being', b'warm', b'and', b'comfortable',
         b'on', b'the', b'sette', b'and', b'having', b'j

In [17]:
# Construire le vocabulaire 
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

# Créer la table de lookup
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

for X_batch, y_batch in train_set.take(1):
    print(X_batch)
    print(y_batch)

tf.Tensor(
[[  22   11   28 ...    0    0    0]
 [   6   21   70 ...    0    0    0]
 [4099 6881    1 ...    0    0    0]
 ...
 [  22   12  118 ...  331 1047    0]
 [1757 4101  451 ...    0    0    0]
 [3365 4392    6 ...    0    0    0]], shape=(32, 60), dtype=int64)
tf.Tensor([0 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0], shape=(32,), dtype=int64)


In [21]:
train_set = datasets["train"].repeat().batch(32).map(preprocess).map(encode_words).prefetch(1)
# valid_set = datasets["validation"].batch(32).map(preprocess).map(encode_words).prefetch(1)
test_set = datasets["test"].batch(32).map(preprocess).map(encode_words).prefetch(1)



In [22]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, validation_data=test_set, steps_per_epoch=train_size // 32, epochs=5)

Epoch 1/5
781/781 [==============================] - 22s 22ms/step - loss: 0.6066 - accuracy: 0.6383 - val_loss: 0.4730 - val_accuracy: 0.7727
Epoch 2/5
781/781 [==============================] - 16s 21ms/step - loss: 0.3685 - accuracy: 0.8419 - val_loss: 0.5046 - val_accuracy: 0.7641
Epoch 3/5
781/781 [==============================] - 17s 22ms/step - loss: 0.2106 - accuracy: 0.9207 - val_loss: 0.6346 - val_accuracy: 0.7437
Epoch 4/5
781/781 [==============================] - 17s 22ms/step - loss: 0.1391 - accuracy: 0.9511 - val_loss: 0.7298 - val_accuracy: 0.7336
Epoch 5/5
781/781 [==============================] - 17s 22ms/step - loss: 0.1067 - accuracy: 0.9617 - val_loss: 0.9589 - val_accuracy: 0.7261


# Réutiliser des plongements pré-entrainés

In [27]:
import tensorflow_hub as hub

model = keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                   dtype=tf.string, input_shape=[], output_shape=[50]),
    # On ne peut plus utiliser de cellules GRU ou LSTM car nous n'avons plus la dimension temporelle ici, chaque phrase étant encodée par un seul vecteur.
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].repeat().batch(32).prefetch(1)
test_set = datasets["test"].batch(32).prefetch(1)
history = model.fit(train_set, validation_data=test_set, steps_per_epoch=train_size // batch_size, epochs=5)

Epoch 1/5
781/781 [==============================] - 5s 5ms/step - loss: 0.5876 - accuracy: 0.6892 - val_loss: 0.5264 - val_accuracy: 0.7369
Epoch 2/5
781/781 [==============================] - 5s 6ms/step - loss: 0.5200 - accuracy: 0.7428 - val_loss: 0.5192 - val_accuracy: 0.7414
Epoch 3/5
781/781 [==============================] - 5s 7ms/step - loss: 0.5137 - accuracy: 0.7469 - val_loss: 0.5155 - val_accuracy: 0.7460
Epoch 4/5
781/781 [==============================] - 5s 6ms/step - loss: 0.5095 - accuracy: 0.7512 - val_loss: 0.5130 - val_accuracy: 0.7470
Epoch 5/5
781/781 [==============================] - 5s 6ms/step - loss: 0.5062 - accuracy: 0.7506 - val_loss: 0.5110 - val_accuracy: 0.7475
